In [4]:
# library import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline
import seaborn as sns
import dask
import dask.array as da
import dask.dataframe as dd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
import os
import re
import glob
import shutil
import gc
from pathlib import Path
from tqdm import tqdm
# showing module
from IPython.display import display

# output display option adjustment
# precision of floating point in numpy
np.set_printoptions(suppress=True, precision=4)

# precision of floating point in pandas
pd.options.display.float_format = '{:.4f}'.format

# display all columns in dataframe
pd.set_option("display.max_columns",None)

# default font size in graph
plt.rcParams["font.size"] = 14

# graph display
sns.set(rc={'figure.figsize':(12,5)});
plt.figure(figsize=(12,5));

# random seed
random_seed = 45

ModuleNotFoundError: No module named 'plotly'

In [1]:
# 以下のカラムは頻出と考えられ、毎回入力するのはめんどくさいので、ポップアップされるように定義します
customer_ID = 'customer_ID'
TARGET = 'target'

# 今回のコンペに関して

### データの読み込みに関して
* 形式を変えたデータセットをpandasで読み込む
  * [`feather`形式](https://www.kaggle.com/datasets/munumbutt/amexfeather) or [`Parquet`形式](https://www.kaggle.com/datasets/odins0n/amex-parquet)
* [`pyspark`](https://www.kaggle.com/code/rakkaalhazimi/export-large-dataset-to-spark) or [`dask`](https://docs.dask.org/en/latest/dataframe.html) で読み込む

### サイズが大きい場合の対処方法

[> How to Work with BIG Datasets on 16G RAM (+Dask)](https://www.kaggle.com/code/yuliagm/how-to-work-with-big-datasets-on-16g-ram-dask)

上記notebookの内容概要
* TIP 1 - 使用していない変数を [`delete`](https://www.sejuku.net/blog/74447) するか & gc.collect()` で[ガベージコレクション](https://techacademy.jp/magazine/19437)(不要になったメモリ領域を開放して再利用する機能)をする
* TIP 2 - データセット内の各カラムのデータタイプを予め定義しておく
  * eg: 本来 float64 だったものを -> float32 と定義してサイズを圧縮する
* TIP 3 - 読み込むデータセットを選択する (including generating your own subsamples)
  * 読み込む行数を選ぶ
  * (`skiprows`) で[読み飛ばす行数を指定する](https://bit.ly/3O90Ze7)
  * 読み飛ばすリストを作成して、読み飛ばす（以下例）
    * ```
      skiplines = np.random.choice(np.arange(1, lines), size=lines-1-1000000, replace=False)
      skiplines=np.sort(skiplines)
      train = pd.read_csv('../input/train.csv', skiprows=skiplines, dtype=dtypes)
      ```
* TIP 4 - バッチ処理をする
  * ひとまとまりのデータに対して、一連の処理を連続で実行する処理方式のこと。大きなデータに関しても、設定したデータ数(チャンク)ごとに処理をする
* TIP 5 - 特定のカラムのみ `import` する
  * 450,000行 × 150カラム より 100万行 × 2カラム の方がメモリ消費が少ないことは容易に想像できる
* TIP 6 - groupby などの処理をするときも一部カラムの一部データのみにするˆ
* TIP 7 - `Dask` を使用する。
  * [DaskについてのQiita記事](https://qiita.com/simonritchie/items/e174f243bc03fb25462e)

## コンペの概要
* コンペ名：[American Express - Default Prediction](https://www.kaggle.com/competitions/amex-default-prediction)

* 目的：毎月の顧客プロファイルから、顧客がクレジットカードの残高分を将来返済しない確率を予測すること
  * ターゲットのバイナリ変数は、最新のクレジットカード明細書から18ヶ月間のパフォーマンスウィンドウを観察することによって計算され、顧客が最新の明細書の日付から120日以内に支払額を支払わない場合、デフォルトとみなされる。

* 評価方法(Evaluation)
  * このコンペではクレカのデフォルト率を予測する。サブミットはちょっと特殊な評価方法で評価される。以下の通り:
    * ```
      M = 0.5*(G+D)  (*G = 正規化ジニ係数, D = デフォルト率 4% )
      ```
      機械学習における `正規化ジニ係数` は経済学などで使用される ジニ係数とは違うので注意。
      * [機械学習のモデル評価、説明可能性のための指標　その１。ジニ係数とAUC](https://qiita.com/Derek/items/4ded249f7a75f8da176c)
      * [DataRobot](https://docs.datarobot.com/ja/docs/modeling/reference/model-detail/opt-metric.html#gini-coefficient)
      * [GINI and AUC relationship](https://stats.stackexchange.com/questions/342329/gini-and-auc-relationship)
      * [Why use Normalized Gini Score instead of AUC as evaluation?](https://stats.stackexchange.com/questions/306287/why-use-normalized-gini-score-instead-of-auc-as-evaluation)

# データ読み込みに関して

> `train data` と `test data` を同時にメモリに載せるとクラッシュする可能性があるため、\
`train data` と `test data` は別々にメモリに乗っけて特徴量生成すると良いかも

前記の通り、データサイズが大きく、安易に `pandas` などを使ってもローカルではメモリが足りないので、ここではcsv形式を `parquet` 形式に変えてimport する
* [該当データダウンロードページ](https://www.kaggle.com/datasets/odins0n/amex-parquet)
* [Load Parquet Files with Low Memory](https://www.kaggle.com/code/odins0n/load-parquet-files-with-low-memory)
* [parquetデータを使用したEDA](https://www.kaggle.com/code/odins0n/amex-default-prediction-detailed-eda)

# 各テーブルの概説

* データについて
  * データセットには各顧客の特徴が各明細書日付ごとに集約されたものが含まれている。特徴は匿名化され、正規化されており、以下のカテゴリに分類されている（カラムの prefix を見ると分かる）:
    * D_*: Delinquency variables
    * S_*: Spend variables
    * P_*: Payment variables
    * B_*: Balance variables
    * R_*: Risk variables
  * 以下のカラムはカテゴリ変数である:
    * B_30, B_38, D_63, D_64, D_66, D_68, D_114, D_116, D_117, D_120, D_126

# EDA (データ確認)

> Daskを使えたらいいなあ

* [PythonのDaskをしっかり調べてみた（大きなデータセットを快適に扱う）](https://qiita.com/simonritchie/items/e174f243bc03fb25462e)

> EDA はこの辺もまずは参考にしてみる
* [AMEX EDA (Comparison of training and test data)](https://www.kaggle.com/code/onodera1/amex-eda-comparison-of-training-and-test-data)

## train data

In [3]:
# parquet データを import
train = pd.read_parquet('../input/amex-data-integer-dtypes-parquet-format/train.parquet')

In [4]:
train.info()

In [5]:
# カラム名をわかりやすくする
titles=['Delinquency '+str(i).split('_')[1] if i.startswith('D') 
        else 'Spend '+str(i).split('_')[1] if i.startswith('S') 
        else 'Payment '+str(i).split('_')[1] if i.startswith('P') 
        else 'Balance '+str(i).split('_')[1] if i.startswith('B') 
        else 'Risk '+str(i).split('_')[1] if i.startswith('R')
        else customer_ID
        for i in train.columns[:-1]
        ]

In [6]:
len(titles)

In [7]:
# train データのカラム名変更
titles.append('target')
train.columns = titles

train dataから作成したカラム変更のためのリストをcsvファイルとして保存する\
-> csvからリストを読み込むことで `train data`をメモリに載せて毎回作らずとも`test data`のカラム名を変えることができる

In [ ]:
# # 以下だとkaggle notebookでは動作しないため、追記する必要がある
# import csv
# # 設定したアウトプットディレクトリに名前のリストを入れる
# f = open(os.path.join(OUTPUT_DIR, 'train_df_col_names.csv'), 'w')
# writer = csv.writer(f)
# writer.writerow(titles)
# f.close()

## train_labels 予測対象

In [8]:
train_labels = pd.read_csv('../input/amexdata/train_labels.csv')

In [9]:
# train dataで予測するID数は 458193 個らしい
train_labels.info()

# 前処理 + 特徴量エンジニアリング

クラッシュするため、各IDごとのレコード数を減らす

In [10]:
# train データに関して、各IDごと2レコードのみにして、プロットするために category カラムを追加
train2 = train.groupby(customer_ID).tail(2)

In [11]:
# メモリ節約のために `train` dataを削除する
del train

# ガベージコレクション
gc.collect()

欠損値率95%以上のカラムを削除する

In [12]:
miss_val = ['Delinquency 87', 'Delinquency 88', 'Delinquency 108', 'Delinquency 111', 'Delinquency 110', 'Balance 39', 'Delinquency 73', 'Balance 42']

train2 = train2.drop(columns=miss_val, axis=1)

## 基本統計量の計算

In [15]:
# 改めて連続変数とカテゴリ変数のリスト作成
COLS = list(train2.columns[2:])
cat_cols = ['Balance 30', 'Balance 38', 'Delinquency 63', 'Delinquency 64', 'Delinquency 66', 'Delinquency 68',
          'Delinquency 114', 'Delinquency 116', 'Delinquency 117', 'Delinquency 120', 'Delinquency 126']
# カテゴリー変数に加えて、customer_ID, timestamp のカラムは除く
con_cols = [col for col in COLS if col not in cat_cols and col not in ['Spend 2', customer_ID]]

In [16]:
# 平均・分散・最小・最大値・最大・最後の値
train_num_agg = train2.groupby(customer_ID)[con_cols].agg(['mean', 'std', 'min', 'max', 'last'])
# マルチカラムになっているのでそれをシングルカラムにしてあげる
## 参考記事：(https://qiita.com/rinascimento741/items/e2fceb8626ac97ebf49b)
train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
train_num_agg.reset_index(inplace = True)

# customer_IDごとの各カテゴリーに該当する数、最後の値
train_cat_agg = train2.groupby(customer_ID)[cat_cols].agg(['count', 'last', 'nunique'])
# マルチカラムになっているのでそれをシングルカラムにしてあげる
train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
train_cat_agg.reset_index(inplace = True)

In [17]:
# 進捗状況を可視化するメソッドをdataframe オブジェクトに追加する（1GBメモリ節約できる）
tqdm.pandas()
# 計算コスト削減のためにfloat64をfloat32に変換
cols = list(train_num_agg.dtypes[train_num_agg.dtypes == 'float64'].index)
train_num_agg.loc[:,cols] = train_num_agg.loc[:,cols].progress_apply(lambda x: x.astype(np.float32))

# 計算コスト削減のためにint64をint32に変換
cols = list(train_cat_agg.dtypes[train_cat_agg.dtypes == 'int64'].index)
train_cat_agg.loc[:,cols] = train_cat_agg.loc[:,cols].progress_apply(lambda x: x.astype(np.int32))

## 差分の計算

In [18]:
train_diff = train2.loc[:,con_cols+[customer_ID]].groupby([customer_ID]).progress_apply(lambda x:np.diff(x.values[-2:,:], axis=0).squeeze().astype(np.float32))
index = train_diff.index
cols = [col + '_diff1' for col in train2[con_cols].columns]
train_diff = pd.DataFrame(train_diff.values.tolist(), columns = cols)
train_diff[customer_ID] = index

以下の方法はnotebookに記載されていた方法だが、若干重い。自分のローカルPCではCPUかRAMが足りずクラッシュしてしまう


In [ ]:
def get_difference(data, num_features):
    df1 = []
    customer_ids = []
    # `tqdm` customer_IDごとでグループ分けをした groupby オブジェクトをcustomer_ID, dfに分ける
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        df1.append(
            # 各変数 ごとに同じcustomer_ID内で前のレコードとの差分を出して、そのうち一番最後のレコードを取得する
            df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        )
        customer_ids.append(customer_id)
    # よくわからん
    df1 = np.concatenate(df1, axis = 0)
    # カラム名に `_diff1`と付けた上で、dataframe型に変える
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    # customer_IDカラムを追加する
    df1['customer_ID'] = customer_ids
    return df1

## マージ

In [19]:
train_num_agg.info()

In [20]:
train_cat_agg.info()

In [21]:
train_diff.info()

In [22]:
# train_num_agg, train_cat_agg, train_diff, train_labels をマージする
train2 = pd.merge(train_num_agg, train_cat_agg, how='inner', on=customer_ID).merge(train_diff, how='inner', on=customer_ID).merge(train_labels, how='inner', on=customer_ID)

In [ ]:
train2.info()

# データ書き出し

In [ ]:
# カレントワーキングディレクトリに保存
train2.to_parquet('/kaggle/working/train2.parquet')

ちなみに以下のコードでコミットなしでファイルをローカルにダウンロードするリンクを作ってくれる

[参考記事]('https://www.currypurin.com/entry/2019/07/19/230524')

In [ ]:
from IPython.display import FileLink
FileLink('train2.parquet')

メモリ食っている変数を探し出して削除

In [ ]:
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 1000:
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

In [ ]:
# メモリ節約のために削除
del train_num_agg, train_cat_agg, train_diff, index, cols, train2, train_labels
gc.collect()

# test data

In [ ]:
# メモリ6GB 近く使用
test = pd.read_parquet('../input/amex-data-integer-dtypes-parquet-format/test.parquet')
%time

In [ ]:
test.info()

train dataから作成したカラム名のリストデータをcsvデータから読み取る

In [ ]:
# kaggle に保存したカラム名リストのcsvデータを開ける
titles = []
import csv
with open("../input/train-df-col-names/train_df_names.csv") as f:
  for row in csv.reader(f):
    titles.append(row)
    print(f"{type(row)}")

In [ ]:
# 多重リストになっていたのを平坦化する
import itertools
titles = list(itertools.chain.from_iterable(titles))

In [ ]:
# test データのカラム名変更
test.columns = titles[:-1]

# 前処理 + 特徴量エンジニアリング

クラッシュするのを防ぐために各IDごと2レコードに絞る

In [ ]:
# test データに関して、各IDごと2レコードのみにして、プロットするために category カラムを追加
test2 = test.groupby(customer_ID).tail(2)

In [ ]:
# メモリ節約のために `test` dataを削除してガベージコレクションする
del test
gc.collect()

In [ ]:
test2.info()

欠損値率95%以上のカラムを削除する

In [ ]:
miss_val = ['Delinquency 87', 'Delinquency 88', 'Delinquency 108', 'Delinquency 111', 'Delinquency 110', 'Balance 39', 'Delinquency 73', 'Balance 42']

test2 = test2.drop(columns=miss_val, axis=1)

このままだとクラッシュする恐れがあるため `dataframe` を分割する

## 基本統計量の計算

In [ ]:
# カテゴリ変数のリスト作成
cat_cols = ['Balance 30', 'Balance 38', 'Delinquency 63', 'Delinquency 64', 'Delinquency 66', 'Delinquency 68',
          'Delinquency 114', 'Delinquency 116', 'Delinquency 117', 'Delinquency 120', 'Delinquency 126']
# drop columns したので改めてカラムリストを作る
COLS = list(test2.columns[2:])
# カテゴリー変数に加えて、customer_ID, timestamp のカラムは除く
con_cols = [col for col in COLS if col not in cat_cols and col not in ['Spend 2', customer_ID]]

ここで一時的にメモリをoveruseしかけるため、改善余地あり

In [ ]:
# 平均・分散・最小・最大値・最大・最後の値
test_num_agg = test2.groupby(customer_ID)[con_cols].agg(['mean', 'std', 'min', 'max', 'last'])
# マルチカラムになっているのでそれをシングルカラムにしてあげる
## 参考記事：(https://qiita.com/rinascimento741/items/e2fceb8626ac97ebf49b)
test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
test_num_agg.reset_index(inplace = True)

# customer_IDごとの各カテゴリーに該当する数、最後の値
test_cat_agg = test2.groupby(customer_ID)[cat_cols].agg(['count', 'last', 'nunique'])
# マルチカラムになっているのでそれをシングルカラムにしてあげる
test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
test_cat_agg.reset_index(inplace = True)

In [ ]:
# 進捗状況を可視化するメソッドをdataframe オブジェクトに追加する
tqdm.pandas()
# 計算コスト削減のためにfloat64をfloat32に変換
cols = list(test_num_agg.dtypes[test_num_agg.dtypes == 'float64'].index)
test_num_agg.loc[:,cols] = test_num_agg.loc[:,cols].progress_apply(lambda x: x.astype(np.float32))

# 計算コスト削減のためにint64をint32に変換
cols = list(test_cat_agg.dtypes[test_cat_agg.dtypes == 'int64'].index)
test_cat_agg.loc[:,cols] = test_cat_agg.loc[:,cols].progress_apply(lambda x: x.astype(np.int32))

## 差分の計算

In [ ]:
test_diff = test2.loc[:,con_cols+[customer_ID]].groupby([customer_ID]).progress_apply(lambda x:np.diff(x.values[-2:,:], axis=0).squeeze().astype(np.float32))
index = test_diff.index
# column rename
cols = [col + '_diff1' for col in test2[con_cols].columns]
test_diff = pd.DataFrame(test_diff.values.tolist(), columns = cols)
test_diff[customer_ID] = index

## マージ

In [ ]:
# test_num_agg, test_cat_agg, test_diff をマージする
test2 = pd.merge(test_num_agg, test_cat_agg, how='inner', on=customer_ID).merge(test_diff, how='inner', on=customer_ID)

del test_num_agg, test_cat_agg, test_diff, index
gc.collect()

In [ ]:
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 1000:
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

## データの保存

In [ ]:
# カレントワーキングディレクトリに保存（notebookリセットしたらなくなる?）
test2.to_parquet('/kaggle/working/test2.parquet')

In [ ]:
# ちなみにこれでコミットなしでファイルをローカルにダウンロードするリンクを作ってくれる
from IPython.display import FileLink
FileLink('test2.parquet')

In [ ]:
del test2
gc.collect()

## カテゴリ変数の変換

カテゴリデータは基本的にそのまま特徴量として扱えないので、数値化する

* One-Hot Encoding -> gbdt系以外（線形モデル etc..）におすすめ
* Label Encoding-> gbdt系 にもおすすめ\
[【sklearn】LabelEncoderの使い方を丁寧に](https://gotutiyan.hatenablog.com/entry/2020/09/08/122621)
* Target Encoding -> gbdt系にはより効果的らしい\
[Target Encoding はなぜ有効なのか](https://speakerdeck.com/hakubishin3/target-encoding-hanazeyou-xiao-nafalseka)

## データマージ

# 学習・予測・サブミットファイル作成

## モデル構築

## モデル評価

## サブミットファイル作成